In [8]:
from nltk.util import ngrams
from itertools import combinations
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import hanlp

In [1]:
# 📌 1. HanLP 토큰화 모델 로드
segmenter = hanlp.load("CTB9_TOK_ELECTRA_BASE_CRF")


100% 359.3 MiB 239.8 KiB/s ETA:  0 s [=========================================]
Decompressing /Users/gangjisu/.hanlp/thirdparty/download.hanlp.com/tok/extra/ctb9_tok_electra_base_crf_20220426_161255.zip to /Users/gangjisu/.hanlp/thirdparty/download.hanlp.com/tok/extra
100%  26.7 KiB  26.7 KiB/s ETA:  0 s [=========================================]
Decompressing /Users/gangjisu/.hanlp/utils/char_table_20210602_202632.json.zip to /Users/gangjisu/.hanlp/utils
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing /Users/gangjisu/.hanlp/transformers/electra_zh_base_20210706_125233.zip to /Users/gangjisu/.hanlp/transformers


In [73]:
 #📌 2. 불용어 리스트 정의
chinese_stopwords =  set([
    "的", "了", "在", "是", "和", "与", "也", "有", "对", "就", "以", "将", "要",
    "但", "其", "而", "此", "我们", "他们", "你们", "可以", "但是", "这样", "这个",
    "其中", "其中之一", "包括", "根据", "由于", "通过", "此外", "同时",
    "记者", "新闻", "媒体", "采访", "发表", "宣布", "透露", "介绍", "报道称",
    "指出", "强调", "证实", "承认", "评价", "评论", "提到", "解释", "分析",
    "总结", "预测", "预计", "关注", "反映", "说明", "进一步",
    "今天", "昨天", "明天", "日前", "近日", "本周", "上周", "下周", "目前",
    "现在", "过去", "未来", "今年", "去年", "明年", "此前", "随后", "当地",
    "政府", "机构", "相关", "部门", "官员", "领导人", "代表", "发言人", "部长", "事务", "委员会",
    "一", "二", "三", "这", "不", "可能", "都", "才", "可", "一直", "到", "如果", "带来",
    "十分", "称", "个", "次", "因", "因此", "最", "即", "当天", "因为", "曾",
    "号", "第", "相当", "两", "很", "所以", "各种", "从而", "仍", "为了", "以及",
    "据", "并", "过", "几", "立即", "着", "于", "为", "说", "却", "使", "国", "还",
    "来说", "至", "会", "除了", "被", "外", "若", "更", "已经", "已", "大大", "不仅",
    "能够", "再", "一些", "明确", "作为", "向",  "是否", "积极", "讨论",
    "越来越", "甚至", "出于", "以来", "重启", "近期", "依然", "影响", "之间", "重点",
    "方面", "力度", "报道", "如何", "周年", "文章", "前往", "通讯社", "事情", "外部",
    "社论", "非常", "上午", "时间", "它们", "演讲", "应该", "不断", "结束", "愿意",
    "最后", "目标", "开始", "缺乏", "自己", "调查", "今后", "乃至", "特别", "深化",
    "即将", "类似", "什么", "没有", "方向", "更为", "需要", "做出", "第三",
    "公司", "平台", "游戏", "环球", "时报", "人民", "日报", "新华社"
])
# "月", "日", "年"

In [5]:
# # 📌 3. JSON 데이터 로드
# file_path = "../data/final_0307.json"
# with open(file_path, "r", encoding="utf-8") as f:
#     data = json.load(f)
# 
# # 📌 4. 데이터프레임 변환
# df = pd.DataFrame(data)
# 
# # 📌 5. HanLP로 토큰화 및 불용어 제거
# def tokenize_and_filter(text):
#     tokens = segmenter(text)
#     return " ".join([word for word in tokens if word not in chinese_stopwords])
# 
# df["tokenized_content"] = df["cleaned_content"].apply(tokenize_and_filter)


In [63]:
# 📌 3. JSON 데이터 로드
file_path = "../data/news.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 📌 4. 데이터프레임 변환
df = pd.DataFrame(data)

In [80]:


def remove_english_stopwords_dates(text, chinese_stopwords):
    # 1️⃣ 영어 단어 제거 (단어뿐만 아니라 단독 알파벳도 포함)
    text = re.sub(r'[a-zA-Z]+', '', text)

    # 2️⃣ 날짜 패턴 제거 (예: "5日", "7月")
    text = re.sub(r'\d+[日月]', '', text)

    # 3️⃣ 문자열을 공백 기준으로 분할
    token_list = text.split()

    # 4️⃣ 불용어 제거
    filtered_list = [word for word in token_list if word not in chinese_stopwords]

    return " ".join(filtered_list)

# DataFrame 적용
df["tokenized_content"] = df["tokenized_content"].apply(lambda x: remove_english_stopwords_dates(x, chinese_stopwords))


In [75]:
df

,title,link,date,content,source,quarter,cleaned_content,tokenized_content,filtered_tokens
0,朝鲜试射导弹亚太骤起波澜,https://www.gmw.cn/01gmrb/2006-07/12/content_4...,2006-07-12,7月5日，朝鲜连续发射7枚导弹，引起国际社会震惊。近日，美日在安理会积极推动制裁朝鲜的议案，...,光明网,2006Q3,7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...,7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 积极...,7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 推动...
1,美朝关系开始解冻,https://www.gmw.cn/01gmrb/2007-03/08/content_5...,2007-03-08,被外界视为“破冰之旅”的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常...,光明网,2007Q1,被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 3月 5 6日 纽约 美国 助理 国务卿 希...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 3月 5 6日 纽约 美国 助理 国务卿 希...
2,美日韩间谍云集朝鲜周边空中侦察地面窃听 - 国际经济,http://intl.ce.cn/zj/200809/19/t20080919_16855...,2008-09-19,神秘，这是朝鲜留给外界的印象。 正因为神秘，一些国家的情报机构费尽心机，通过投放卫星、派遣侦...,中国经济网,2008Q3,神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...
3,美日韩紧盯朝鲜射导弹(图) - 国际经济,http://intl.ce.cn/zj/200902/11/t20090211_18171...,2009-02-11,改进型“大浦洞2号”据说可攻击美本土，甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关...,中国经济网,2009Q1,改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...,改进型 大浦洞 2 据说 攻击 美 本土 甚至 具有 搭载 核 弹头 技术 朝鲜 最近 频繁...,改进型 大浦洞 2 据说 攻击 美 本土 具有 搭载 核 弹头 技术 朝鲜 最近 频繁 发出...
4,美国朝鲜问题特使称美愿与朝对话,https://world.huanqiu.com/article/9CaKrnJlEWs,2009-03-07,新华网首尔3月7日电 (记者李拯宇 干玉兰) 美国朝鲜问题特使斯蒂芬·博斯沃思7日在韩国说，...,环球网,2009Q1,新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...,新华网 首尔 3月 7日 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 7日 ...,新华网 首尔 3月 7日 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 7日 ...
...,...,...,...,...,...,...,...,...,...
1034,詹德斌：韩国政局变动，美国要做的不应是施压,https://hqtime.huanqiu.com/share/article/4Ky1A...,2025-01-07,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅，但防止韩国“后弹劾政局”...,环球网,2025Q1,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅但防止韩国后弹劾政局对美韩...,美国 国务卿 布林肯 1月 5日 抵达 韩国 。 看上去 似乎 礼仪性 道别 之 旅 防止 ...,美国 国务卿 布林肯 1月 5日 抵达 韩国 。 看上去 似乎 礼仪性 道别 之 旅 防止 ...
1035,朝鲜谴责美韩军事挑衅导致地区局势恶化,http://www.xinhuanet.com/20250126/55a39b230ccf...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,新华网,2025Q1,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,新华社 首尔 1月 26日 电 朝中社 26日 报道 朝鲜 外务省 对外 政策 室长 谈话 ...,首尔 1月 26日 电 朝中社 26日 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 ...
1036,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,光明网,2025Q1,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,中新网 1月 26日 电 朝中社 报道 时间 1月 26日 朝鲜 外务省 对外 政策 室长 ...,中新网 1月 26日 电 朝中社 1月 26日 朝鲜 外务省 对外 政策 室长 谈话 谴责 ...
1037,石破茂与特朗普会面，美日“小集团”谋划“大算盘” | 国际识局,http://intl.ce.cn/qqss/202502/09/t20250209_392...,2025-02-09,中新网2月9日电(记者 孟湘君)特朗普当选新一任美国总统后，意大利、以色列等国领导人先后与其...,中国经济网,2025Q1,中新网2月9日电记者 孟湘君特朗普当选新一任美国总统后意大利以色列等国领导人先后与其会面。近...,中新网 2月 9日 电 孟湘君 特朗普 当选 新 任 美国 总统 后 意大利 以色列 先后 ...,中新网 2月 9日 电 孟湘君 特朗普 当选 新 任 美国 总统 后 意大利 以色列 先后 ...


'7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 积极 推动 制裁 朝鲜 议案 加紧 协调 行动 方案 。 朝鲜 举 自身 推向 国际 政治 风云 风口浪尖 。 朝鲜 朝鲜 发射 导弹 给 东北亚 地区 局势 增加 更加 复杂 难以 预料 因素 。 日本 提前 部署 爱国者3 地空 拦截 导弹 。 韩国 担忧 日本 借口 趁机 加速 军事化 步伐 。 日本 投入 12亿 美元 用 建立 导弹 防御 系统 并且 日 美 联合 计划 部分 。 日本 开始 从 美国 采购 武器 。 日本 计划 2010年 太空 发射 八 颗 间谍 卫星 用 配合 导弹 防御 系统 。 朝鲜 发射 导弹 后 美 日 韩 澳 俄 英 法 纷纷 声明 朝鲜 举 挑衅 行为 破坏 亚太 稳定 。 尤其 美 日 反应 强烈 事态 升级 武装 冲突 可能性 大 。 日 美 安理会 推动 制裁 议案 几率 微乎其微 。 人 朝鲜 急切 想 试射 导弹 达成 目的 想法 适得其反 美国 收紧 朝 孤立 政策 日 朝 关系 正常化 更为 困难 。 日本 考虑 单方面 朝鲜 实施 经济 制裁 在内 反制 措施 。 朝鲜 问题 只有 各 方 表现 出 诚意 建设性 危机 能 化为 解决 问题 契机 。'


In [81]:
# 📌 6. TF-IDF 벡터라이저 적용
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(df["tokenized_content"])

# 📌 7. 단어 리스트 가져오기
feature_names = vectorizer.get_feature_names_out()

# 📌 8. 기사별 상위 중요 키워드 및 점수 추출 함수
def get_top_keywords(row, top_n=10):
    row_array = row.toarray().flatten()  # 벡터를 1차원 배열로 변환
    indices = np.argsort(row_array)[::-1]  # 중요도 높은 순 정렬
    top_indices = indices[:top_n]
    top_words = [(feature_names[i], row_array[i]) for i in top_indices]  # (단어, TF-IDF 점수) 저장
    return ", ".join([f"{word} ({score:.4f})" for word, score in top_words])

# 📌 9. 기사별 키워드 및 점수 추출하여 저장
df["keywords"] = [get_top_keywords(row) for row in tfidf_matrix]


# 📌 11. 결과 확인 (상위 10개 기사)
print("\n✅ 기사별 TF-IDF 키워드 상위 10개:")
print(df[["cleaned_content", "tokenized_content", "keywords"]].head(10))



✅ 기사별 TF-IDF 키워드 상위 10개:
                                     cleaned_content  \
0  7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...   
1  被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...   
2  神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...   
3  改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...   
4  新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...   
5  环球时报驻韩国特约记者沈松报道据韩联社报道朝鲜人民军总参谋部2日对于韩美日3国就朝鲜远程火箭...   
6  中新网4月4日电 综合媒体报道4月4日朝鲜周边紧张态势骤然加剧朝鲜官方已表示即将发射卫星日韩...   
7  新华网北京４月１４日电 综合新华社驻外记者报道联合国安理会１３日就朝鲜发射问题通过主席声明后...   
8  新华网平壤４月１８日电记者高浩荣张滨阳朝鲜人民军总参谋部发言人１８日在平壤发表谈话说美国日本...   
9  中国国产航母及舰载机想像图 澳大利亚悉尼先驱晨报4月29日文章原题我们的问题中国是敌是友？ ...   

                                   tokenized_content  \
0  朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 推动 制裁 朝鲜...   
1  外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 5 纽约 美国 助理 国务卿 希尔 关系 正...   
2  神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...   
3  改进型 大浦洞 2 据说 攻击 美 本土 具有 搭载 核 弹头 技术 朝鲜 最近 频繁 发出...   
4  新华网 首尔 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 韩国 美国 朝鲜 ...   
5  驻 韩国 特约 沈松 韩联社 朝鲜 

In [78]:
df

,title,link,date,content,source,quarter,cleaned_content,tokenized_content,filtered_tokens,keywords
0,朝鲜试射导弹亚太骤起波澜,https://www.gmw.cn/01gmrb/2006-07/12/content_4...,2006-07-12,7月5日，朝鲜连续发射7枚导弹，引起国际社会震惊。近日，美日在安理会积极推动制裁朝鲜的议案，...,光明网,2006Q3,7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...,7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 推动...,7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 推动...,"朝鲜 (0.3892), 导弹 (0.3622), 发射 (0.2870), 制裁 (0.2..."
1,美朝关系开始解冻,https://www.gmw.cn/01gmrb/2007-03/08/content_5...,2007-03-08,被外界视为“破冰之旅”的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常...,光明网,2007Q1,被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 3月 5 6日 纽约 美国 助理 国务卿 希...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 3月 5 6日 纽约 美国 助理 国务卿 希...,"会谈 (0.5107), 朝鲜 (0.2910), 关系 (0.2469), 美元 (0.2..."
2,美日韩间谍云集朝鲜周边空中侦察地面窃听 - 国际经济,http://intl.ce.cn/zj/200809/19/t20080919_16855...,2008-09-19,神秘，这是朝鲜留给外界的印象。 正因为神秘，一些国家的情报机构费尽心机，通过投放卫星、派遣侦...,中国经济网,2008Q3,神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...,"情报 (0.5933), 朝鲜 (0.5468), 侦察 (0.2270), 卫星 (0.1..."
3,美日韩紧盯朝鲜射导弹(图) - 国际经济,http://intl.ce.cn/zj/200902/11/t20090211_18171...,2009-02-11,改进型“大浦洞2号”据说可攻击美本土，甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关...,中国经济网,2009Q1,改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...,改进型 大浦洞 2 据说 攻击 美 本土 具有 搭载 核 弹头 技术 朝鲜 最近 频繁 发出...,改进型 大浦洞 2 据说 攻击 美 本土 具有 搭载 核 弹头 技术 朝鲜 最近 频繁 发出...,"导弹 (0.6392), 朝鲜 (0.4414), 试射 (0.2939), 公里 (0.2..."
4,美国朝鲜问题特使称美愿与朝对话,https://world.huanqiu.com/article/9CaKrnJlEWs,2009-03-07,新华网首尔3月7日电 (记者李拯宇 干玉兰) 美国朝鲜问题特使斯蒂芬·博斯沃思7日在韩国说，...,环球网,2009Q1,新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...,新华网 首尔 3月 7日 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 7日 ...,新华网 首尔 3月 7日 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 7日 ...,"7日 (0.6547), 朝鲜 (0.4015), 对话 (0.2834), 美国 (0.2..."
...,...,...,...,...,...,...,...,...,...,...
1034,詹德斌：韩国政局变动，美国要做的不应是施压,https://hqtime.huanqiu.com/share/article/4Ky1A...,2025-01-07,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅，但防止韩国“后弹劾政局”...,环球网,2025Q1,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅但防止韩国后弹劾政局对美韩...,美国 国务卿 布林肯 1月 5日 抵达 韩国 。 看上去 似乎 礼仪性 道别 之 旅 防止 ...,美国 国务卿 布林肯 1月 5日 抵达 韩国 。 看上去 似乎 礼仪性 道别 之 旅 防止 ...,"外交 (0.4189), 韩国 (0.2494), 尹锡悦 (0.2271), 价值观 (0..."
1035,朝鲜谴责美韩军事挑衅导致地区局势恶化,http://www.xinhuanet.com/20250126/55a39b230ccf...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,新华网,2025Q1,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,首尔 1月 26日 电 朝中社 26日 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 ...,首尔 1月 26日 电 朝中社 26日 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 ...,"谈话 (0.5149), 行为 (0.2825), 朝方 (0.2668), 挑衅 (0.2..."
1036,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,光明网,2025Q1,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,中新网 1月 26日 电 朝中社 1月 26日 朝鲜 外务省 对外 政策 室长 谈话 谴责 ...,中新网 1月 26日 电 朝中社 1月 26日 朝鲜 外务省 对外 政策 室长 谈话 谴责 ...,"谈话 (0.5808), 行为 (0.2656), 朝方 (0.2508), 挑衅 (0.1..."
1037,石破茂与特朗普会面，美日“小集团”谋划“大算盘” | 国际识局,http://intl.ce.cn/qqss/202502/09/t20250209_392...,2025-02-09,中新网2月9日电(记者 孟湘君)特朗普当选新一任美国总统后，意大利、以色列等国领导人先后与其...,中国经济网,2025Q1,中新网2月9日电记者 孟湘君特朗普当选新一任美国总统后意大利以色列等国领导人先后与其会面。近...,中新网 2月 9日 电 孟湘君 特朗普 当选 新 任 美国 总统 后 意大利 以色列 先后 ...,中新网 2月 9日 电 孟湘君 特朗普 当选 新 任 美国 总统 后 意大利 以色列 先后 ...,"特朗普 (0.5640), 科技 (0.2361), 日本 (0.1956), 同盟 (0...."


In [55]:
import re
import pandas as pd
from itertools import permutations, combinations
from nltk.util import ngrams

# 📌 (단어, 점수) 형식의 문자열을 리스트로 변환하는 함수
def parse_keywords(keyword_str):
    if isinstance(keyword_str, str):  # 문자열일 경우 변환
        keyword_list = re.findall(r"([\u4e00-\u9fa5]+) \((\d+\.\d+)\)", keyword_str)
        return [(word, float(score)) for word, score in keyword_list]
    return keyword_str  # 이미 리스트인 경우 그대로 반환

# 📌 N-gram 생성 함수 (슬라이딩 윈도우, 순열, 조합 지원)
def generate_ngrams(keywords, n=2, method="sliding_window"):
    if not keywords or not isinstance(keywords, list):  # 빈 값 처리
        return []

    if not all(isinstance(word[0], str) for word in keywords):  # ✅ 단어가 문자열인지 확인
        return []

    words = [word[0] for word in keywords]  # ✅ 단어만 추출 (점수 제거)

    if method == "sliding_window":
        from nltk.util import ngrams
        return [tuple(ngram) for ngram in ngrams(words, n)]

    elif method == "permutations":
        return [tuple(ngram) for ngram in permutations(words, n)]

    elif method == "combinations":
        return [tuple(ngram) for ngram in combinations(words, n)]

    return []




# 📌 키워드 컬럼 변환
df["keywords"] = df["keywords"].apply(parse_keywords)

# 🔹 **n=2 (Bigrams)**
df["bigrams_sw"] = df["keywords"].apply(lambda x: generate_ngrams(x, 2, method="sliding_window"))
df["bigrams_perm"] = df["keywords"].apply(lambda x: generate_ngrams(x, 2, method="permutations"))
df["bigrams_comb"] = df["keywords"].apply(lambda x: generate_ngrams(x, 2, method="combinations"))

# 🔹 **n=3 (Trigrams)**
df["trigrams_sw"] = df["keywords"].apply(lambda x: generate_ngrams(x, 3, method="sliding_window"))
df["trigrams_perm"] = df["keywords"].apply(lambda x: generate_ngrams(x, 3, method="permutations"))
df["trigrams_comb"] = df["keywords"].apply(lambda x: generate_ngrams(x, 3, method="combinations"))


In [82]:
output_file = "../data/news_with_keywords1.json"

# 📌 필요 없는 컬럼 리스트
columns_to_drop = ["sentences", "filtered_sentences", "num_filtered_sentences", "num_sentences", "ratio", "keywords_type", "filtered_tokens"]

# 📌 해당 컬럼 삭제 (존재하지 않아도 에러 발생하지 않도록 `errors="ignore"` 설정)
df.drop(columns=columns_to_drop, inplace=True, errors="ignore")

# 📌 2. JSON 덤프를 위한 데이터 변환
json_data = df.to_dict(orient="records")  # DataFrame → JSON 리스트 변환

# 📌 3. Pretty Print JSON 파일 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)

In [56]:
df

,title,link,date,content,source,quarter,cleaned_content,tokenized_content,keywords,bigrams,trigrams,bigrams_perm,bigrams_sw,bigrams_comb,trigrams_sw,trigrams_perm,trigrams_comb
0,朝鲜试射导弹亚太骤起波澜,https://www.gmw.cn/01gmrb/2006-07/12/content_4...,2006-07-12,7月5日，朝鲜连续发射7枚导弹，引起国际社会震惊。近日，美日在安理会积极推动制裁朝鲜的议案，...,光明网,2006Q3,7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...,7月 5日 朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 积极...,"[[朝鲜, 0.3875], [导弹, 0.3606], [发射, 0.2858], [制裁...","[[朝鲜, 导弹], [导弹, 发射], [发射, 制裁], [制裁, 日本], [日本, ...","[[朝鲜, 导弹, 发射], [导弹, 发射, 制裁], [发射, 制裁, 日本], [制裁...","[(朝鲜, 导弹), (朝鲜, 发射), (朝鲜, 制裁), (朝鲜, 日本), (朝鲜, ...","[(朝鲜, 导弹), (导弹, 发射), (发射, 制裁), (制裁, 日本), (日本, ...","[(朝鲜, 导弹), (朝鲜, 发射), (朝鲜, 制裁), (朝鲜, 日本), (朝鲜, ...","[(朝鲜, 导弹, 发射), (导弹, 发射, 制裁), (发射, 制裁, 日本), (制裁...","[(朝鲜, 导弹, 发射), (朝鲜, 导弹, 制裁), (朝鲜, 导弹, 日本), (朝鲜...","[(朝鲜, 导弹, 发射), (朝鲜, 导弹, 制裁), (朝鲜, 导弹, 日本), (朝鲜..."
1,美朝关系开始解冻,https://www.gmw.cn/01gmrb/2007-03/08/content_5...,2007-03-08,被外界视为“破冰之旅”的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常...,光明网,2007Q1,被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 3月 5 6日 纽约 美国 助理 国务卿 希...,"[[会谈, 0.5007], [朝鲜, 0.2852], [关系, 0.2421], [美元...","[[会谈, 朝鲜], [朝鲜, 关系], [关系, 美元], [美元, 美国], [美国, ...","[[会谈, 朝鲜, 关系], [朝鲜, 关系, 美元], [关系, 美元, 美国], [美元...","[(会谈, 朝鲜), (会谈, 关系), (会谈, 美元), (会谈, 美国), (会谈, ...","[(会谈, 朝鲜), (朝鲜, 关系), (关系, 美元), (美元, 美国), (美国, ...","[(会谈, 朝鲜), (会谈, 关系), (会谈, 美元), (会谈, 美国), (会谈, ...","[(会谈, 朝鲜, 关系), (朝鲜, 关系, 美元), (关系, 美元, 美国), (美元...","[(会谈, 朝鲜, 关系), (会谈, 朝鲜, 美元), (会谈, 朝鲜, 美国), (会谈...","[(会谈, 朝鲜, 关系), (会谈, 朝鲜, 美元), (会谈, 朝鲜, 美国), (会谈..."
2,美日韩间谍云集朝鲜周边空中侦察地面窃听 - 国际经济,http://intl.ce.cn/zj/200809/19/t20080919_16855...,2008-09-19,神秘，这是朝鲜留给外界的印象。 正因为神秘，一些国家的情报机构费尽心机，通过投放卫星、派遣侦...,中国经济网,2008Q3,神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...,"[[情报, 0.5944], [朝鲜, 0.5479], [侦察, 0.2274], [卫星...","[[情报, 朝鲜], [朝鲜, 侦察], [侦察, 卫星], [卫星, 这些], [这些, ...","[[情报, 朝鲜, 侦察], [朝鲜, 侦察, 卫星], [侦察, 卫星, 这些], [卫星...","[(情报, 朝鲜), (情报, 侦察), (情报, 卫星), (情报, 这些), (情报, ...","[(情报, 朝鲜), (朝鲜, 侦察), (侦察, 卫星), (卫星, 这些), (这些, ...","[(情报, 朝鲜), (情报, 侦察), (情报, 卫星), (情报, 这些), (情报, ...","[(情报, 朝鲜, 侦察), (朝鲜, 侦察, 卫星), (侦察, 卫星, 这些), (卫星...","[(情报, 朝鲜, 侦察), (情报, 朝鲜, 卫星), (情报, 朝鲜, 这些), (情报...","[(情报, 朝鲜, 侦察), (情报, 朝鲜, 卫星), (情报, 朝鲜, 这些), (情报..."
3,美日韩紧盯朝鲜射导弹(图) - 国际经济,http://intl.ce.cn/zj/200902/11/t20090211_18171...,2009-02-11,改进型“大浦洞2号”据说可攻击美本土，甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关...,中国经济网,2009Q1,改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...,改进型 大浦洞 2 据说 攻击 美 本土 甚至 具有 搭载 核 弹头 技术 朝鲜 最近 频繁...,"[[导弹, 0.6383], [朝鲜, 0.4407], [试射, 0.2934], [公里...","[[导弹, 朝鲜], [朝鲜, 试射], [试射, 公里], [公里, 射程], [射程, ...","[[导弹, 朝鲜, 试射], [朝鲜, 试射, 公里], [试射, 公里, 射程], [公里...","[(导弹, 朝鲜), (导弹, 试射), (导弹, 公里), (导弹, 射程), (导弹, ...","[(导弹, 朝鲜), (朝鲜, 试射), (试射, 公里), (公里, 射程), (射程, ...","[(导弹, 朝鲜), (导弹, 试射), (导弹, 公里), (导弹, 射程), (导弹, ...","[(导弹, 朝鲜, 试射), (朝鲜, 试射, 公里), (试射, 公里, 射程), (公里...","[(导弹, 朝鲜, 试射), (导弹, 朝鲜, 公里), (导弹, 朝鲜, 射程), (导弹...","[(导弹, 朝鲜, 试射), (导弹, 朝鲜, 公里), (导弹, 朝鲜, 射程), (导弹..."
4,美国朝鲜问题特使称美愿与朝对话,https://world.huanqiu.com/article/9CaKrnJlEWs,2009-03-07,新华网首尔3月7日电 (记者李拯宇 干玉兰) 美国朝鲜问题特使斯蒂芬·博斯沃思7日在韩国说，...,环球网,2009Q1,新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...,新华网 首尔 3月 7日 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 7日 ...,"[[日, 0.6123], [愿意, 0.4147], [朝鲜, 0.3755], [对话,...","[[日, 愿意], [愿意, 朝鲜], [朝鲜, 对话], [对话, 美国], [美国, 接...","[[日, 愿意, 朝鲜], [愿意, 朝鲜, 对话], [朝鲜, 对话, 美国], [对话,...","[(日, 愿意), (日, 朝鲜), (日, 对话), (日, 美国), (日, 接触), ...","[(日, 愿意), (愿意, 朝鲜), (朝鲜, 对话), (对话, 美国), (美国, 接...","[(日, 愿意), (日, 朝鲜), (日, 对话), (日, 美国), (日, 接触), ...","[(日, 愿意, 朝鲜), (愿意, 朝鲜, 对话), (朝鲜, 对话, 美国), (对话,...","[(日, 愿意, 朝鲜), (日, 愿意, 对话), (日, 愿意, 美国), (日, 愿意...","[(日, 愿意, 朝鲜), (日, 愿意, 对话), (日, 愿意, 美国), (日, 愿意..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,詹德斌：韩国政局变动，美国要做的不应是施压,https://hqtime.huanqiu.com/share/article/4Ky1A...,2025-01-07,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅，但防止韩国“后弹劾政局”...,环球网,2025Q1,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅但防止韩国后弹劾政局对美韩...,美国 国务卿

In [54]:
print(df["keywords"].head(10))  # 상위 10개 확인
print(type(df["keywords"].iloc[0]))  # 첫 번째 값의 타입 확인


0    [[朝鲜, 0.3875], [导弹, 0.3606], [发射, 0.2858], [制裁...
1    [[会谈, 0.5007], [朝鲜, 0.2852], [关系, 0.2421], [美元...
2    [[情报, 0.5944], [朝鲜, 0.5479], [侦察, 0.2274], [卫星...
3    [[导弹, 0.6383], [朝鲜, 0.4407], [试射, 0.2934], [公里...
4    [[日, 0.6123], [愿意, 0.4147], [朝鲜, 0.3755], [对话,...
5    [[卫星, 0.5152], [发射, 0.3469], [拦截, 0.3001], [和平...
6    [[发射, 0.5291], [火箭, 0.4553], [日, 0.4221], [朝鲜,...
7    [[安理会, 0.5045], [声明, 0.4419], [主席, 0.3156], [朝...
8    [[军方, 0.3717], [安理会, 0.2972], [朝鲜, 0.2882], [主...
9    [[中国, 0.4311], [澳大利亚, 0.4145], [白皮书, 0.3789], ...
Name: keywords, dtype: object
<class 'list'>


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# 전체 데이터셋에 대해 TF-IDF를 계산 (IDF는 전체 데이터셋에 대해 계산)
def calculate_keywords_by_quarter_with_global_idf(df, quarter_col="input_quarter", content_col="tokenized_content", top_n=10):
    keywords_by_quarter = {}

    # 1. 전체 데이터셋에 대해 TF-IDF 계산 (IDF는 전체 데이터셋을 기준으로 계산)
    # 전체 데이터셋을 기준으로 IDF 값을 계산합니다.
    vectorizer = TfidfVectorizer(max_features=1000)
    vectorizer.fit(df[content_col])  # 전체 데이터셋을 기준으로 IDF 값 계산

    # 2. 분기별로 데이터를 그룹화하여 처리 (분기별로 중요 키워드 추출)
    for quarter, group in df.groupby(quarter_col):
        # 3. 해당 분기 내의 기사를 대상으로 TF-IDF 계산 (IDF는 고정값 사용)
        tfidf_matrix_quarter = vectorizer.transform(group[content_col])  # 분기 내의 TF만 계산

        # 4. 각 분기 내에서 중요한 단어들을 추출하는 함수
        def get_top_keywords_for_quarter(tfidf_matrix_quarter, top_n=10):
            # 분기 내에서 단어들의 TF-IDF 점수를 합산
            sum_tfidf = np.sum(tfidf_matrix_quarter.toarray(), axis=0)  # 분기 내의 단어들의 TF-IDF 점수 합산
            top_indices = np.argsort(sum_tfidf)[::-1][:top_n]  # 중요도 높은 순으로 상위 단어 추출
            top_words = [(vectorizer.get_feature_names_out()[i], sum_tfidf[i]) for i in top_indices]  # (단어, 중요도 점수) 저장
            return ", ".join([f"{word} ({score:.4f})" for word, score in top_words])
        # 5. 해당 분기 내에서 중요한 키워드를 추출
        keywords_by_quarter[quarter] = get_top_keywords_for_quarter(tfidf_matrix_quarter)

    return keywords_by_quarter  # 결과로 분기별 키워드를 저장한 딕셔너리 반환

# 예시로 'input_quarter' 컬럼을 기준으로 그룹화하여 각 분기별로 키워드 계산
keywords_by_quarter = calculate_keywords_by_quarter_with_global_idf(df)

# 6. 모든 분기별 결과 출력
# 각 분기별로 추출된 중요 키워드를 출력합니다.
for quarter, keywords in keywords_by_quarter.items():
    print(f"\n✅ {quarter} 분기의 TF-IDF 키워드 상위 10개:")
    print(keywords)  # 각 분기별로 중요한 키워드 출력
